### Import Package:

In [5]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
import pandas as pd
import os
import rdkit

### Some Functions:

In [34]:
#convert peptide to smi:
def convert_smi(peptide):
    peptide_smile = Chem.MolToSmiles(Chem.MolFromFASTA(peptide))
    return peptide_smile

#convert smi to mol:
def smi2mol(smi):
    mol = Chem.MolFromSmiles(smi)
    m3d = Chem.AddHs(mol)
    return m3d

def ConformerGenerate(mol,peptide,numconfs=1):
    #Conformers generated:
    cids = AllChem.EmbedMultipleConfs(mol, numConfs=numconfs,clearConfs=False)

    #Energy optimization:
    res = AllChem.MMFFOptimizeMoleculeConfs(mol,numThreads=0,maxIters=800) 
    return mol,cids

def generateMol(molname,mol,conflist):
    w = Chem.SDWriter(molname+".sdf")
    count = 0
    for cid in conflist:
        count += 1
        w.write(mol, confId=cid)
    w.close()
    return None

### Import Data：

In [4]:
file = "admetreport.csv"
data = pd.read_csv(file)
data.head()

,pepname,HIA,BBB,hepatotoxicity,nephrotoxicity,score,score.1
0,PPCPW,0.6166,-0.600,-0.5125,-0.8811,0.023640,2.363987
1,VVCTY,0.9193,-0.725,-0.6038,-0.5694,0.015041,1.504145
2,PECEVY,0.7508,-0.625,-0.5058,-0.8558,0.012808,1.280843
3,PCY,0.6448,-0.525,-0.5620,-0.8435,0.010344,1.034445
4,AVCY,0.9215,-0.725,-0.5380,-0.6580,0.010043,1.004287


In [11]:
peptides = list(data["pepname"])

### Loop:

In [36]:
#make save folder:
State = os.path.exists("peptideSDF")
if State == False:
    os.mkdir("peptideSDF")
wd = os.path.abspath(".").replace("\\","\\")+"\\"
nwd = wd+"peptideSDF\\" 
#loop and save:
for i in range(len(peptides)):
    peptide = peptides.pop()
    smi = convert_smi(peptide)
    mol = smi2mol(smi)
    mol,cids = ConformerGenerate(mol,peptide)
    generateMol(peptide,mol,cids)
    os.rename(wd+peptide+".sdf", nwd+peptide+".sdf")
    del mol,cids

### Merge all SDF:

In [28]:
wd = os.path.abspath("peptideSDF").replace("\\","\\")+"\\"
files = os.listdir(wd)
with open("peptides.sdf","w") as wf:
    for f in files:
        if f[-3::] != "sdf":
            continue
        with open(wd+f,"r") as rf:
            data = rf.read()
            rf.close()
        wf.write(f.replace(".sdf",""))
        wf.write(data)
    wf.close()
print("Now, a sdf files had been built, you can check it in a pymol software.")